In [ ]:
!pip install pandas
!pip install --upgrade kagglehub
!pip install -U LibRecommender
!pip install keras==2.12.0 tensorflow==2.12.0

!pip show LibRecommender

In [ ]:
import os
import pandas as pd
from libreco.data import random_split, DatasetPure
from libreco.algorithms import BPR
from libreco.evaluation import evaluate
import kagglehub
import tensorflow as tf

class RecipeRecommender:
    def __init__(self, data_path="shuyangli94/food-com-recipes-and-user-interactions"):
        self.data_path = data_path
        self.model = None
        self.data_info = None
        self.name_df = None
        self.data_filtered = None
        self.train_data = None
        self.eval_data = None
        self.test_data = None
        self.user_id_map = {}
        self._load_recipe_names()

        # Initialize recipe name mapping
        self._load_recipe_names()


    def load_and_preprocess(self, min_interactions):
        """Load and preprocess interaction data"""
        # Download and load dataset
        path = kagglehub.dataset_download(self.data_path)

        # Load and combine interaction data
        train = pd.read_csv(os.path.join(path, "interactions_train.csv"))
        eval = pd.read_csv(os.path.join(path, "interactions_validation.csv"))
        test = pd.read_csv(os.path.join(path, "interactions_test.csv"))

        combined = pd.concat([train, eval, test], ignore_index=True)
        combined = self._rename_and_filter_data(combined)

        # Filter items
        item_counts = combined["item"].value_counts()
        items_to_keep = item_counts[item_counts >= min_interactions].index
        filtered = combined[combined["item"].isin(items_to_keep)]

        # Filter users
        user_counts = filtered["user"].value_counts()
        users_to_keep = user_counts[user_counts >= min_interactions].index
        self.data_filtered = filtered[filtered["user"].isin(users_to_keep)]

    def train(self, embed_size=256, n_epochs=5, lr=5e-5):
        """Train the recommendation model"""
        # Split data
        self.train_data, self.eval_data, self.test_data = random_split(
            self.data_filtered,
            multi_ratios=[0.8, 0.1, 0.1]
        )

        # Build datasets
        self.train_data, self.data_info = DatasetPure.build_trainset(self.train_data)
        self.eval_data = DatasetPure.build_evalset(self.eval_data)
        self.test_data = DatasetPure.build_testset(self.test_data)

        # Initialize model
        tf.compat.v1.reset_default_graph()
        self.model = BPR(
            task="ranking",
            data_info=self.data_info,
            loss_type="bpr",
            embed_size=embed_size,
            n_epochs=n_epochs,
            lr=lr,
            batch_size=1024,
            num_neg=5,
            reg=5e-6,
            sampler="random"
        )

        # Train model
        self.model.fit(
            self.train_data,
            neg_sampling=True,
            shuffle=True,
            verbose=2,
            eval_data=self.eval_data,
            metrics=["loss", "roc_auc", "precision", "recall", "ndcg"]
        )
    def save_recommendations_as_csv(self,items_information,amount_of_recs, path):
      df = self.get_recommendations(items_information,amount_of_recs)
      df.to_csv(path, index=False)
      return df

    def get_recommendations(self, items_information, n_rec):
        """
        Holt Empfehlungen für alle User in user_id_map und speichert die Ergebnisse in einem DataFrame.
        """
        dfs = []
        for user_identifier in self.user_id_map:
            df = self.get_recommendation(user_identifier, n_rec, items_information)
            dfs.append(df)
        # Alle einzelnen DataFrames zusammenfügen
        final_df = pd.concat(dfs, ignore_index=True)
        return final_df

    def get_recommendation(self, user_identifier, n_rec, items_information):
      """Get recommendations for a user (UUID or numeric ID) und speichert alle Daten in einem DataFrame"""
      if not self.model:
          raise ValueError("Model not trained. Call train() first.")

      # UUID Lookup
      if isinstance(user_identifier, str):
          if user_identifier not in self.user_id_map:
              raise ValueError(f"User UUID '{user_identifier}' not found.")
          user_id = self.user_id_map[user_identifier]

     # Empfehlungen abrufen
      recommendations = self.model.recommend_user(
          user=user_id,
          n_rec=n_rec,
          filter_consumed=True
     )

      # Liste für die Daten vorbereiten
      records = []
      for recipe in recommendations[user_id]:
          # Item-Titel und Zutaten anhand der recipe_id abrufen
          item_title, item_ingredients = self.__find_item_by_id(recipe, items_information)
          # Datensatz zur Liste hinzufügen
          records.append({
              "uuid": user_identifier,
              "item_id": recipe,
             "item_title": item_title,
              "item_ingredients": item_ingredients
          })

      # DataFrame aus der Liste erstellen
      df = pd.DataFrame(records)
      return df

    def evaluate(self):
        """Evaluate model performance"""
        return evaluate(
            model=self.model,
            data=self.test_data,
            neg_sampling=True,
            metrics=["loss", "roc_auc", "precision", "recall", "ndcg"]
        )

    def info(self, UUID):
      """Gibt einen DataFrame mit allen Interaktionen des angegebenen Benutzers (UUID) zurück."""
      # Überprüfen, ob Daten geladen wurden
      if self.data_filtered is None or not isinstance(self.data_filtered, pd.DataFrame):
          return pd.DataFrame(columns=["user", "item", "label", "name"])

      # Prüfen, ob die UUID vorhanden ist
      if UUID not in self.user_id_map:
          return pd.DataFrame(columns=["user", "item", "label", "name"])

      # Numerische Benutzer-ID abrufen
      user_id = self.user_id_map[UUID]

      # Interaktionen filtern
      user_interactions = self.data_filtered[self.data_filtered['user'] == user_id].copy()

      if user_interactions.empty:
          return pd.DataFrame(columns=["user", "item", "label", "name"])

      # UUID statt numerischer ID setzen
      user_interactions['user'] = UUID

      # Rezeptnamen hinzufügen
      merged = user_interactions.merge(self.name_df, left_on='item', right_on='id', how='left')
      merged['name'] = merged['name'].fillna('Unknown Recipe')

      # Ergebnis formatieren
      result = merged[['user', 'item', 'label', 'name']]

      return result

    def save(self, storagepath):
      """Speichert Modell und Zustand"""
      if not self.model:
          raise ValueError("Modell nicht trainiert")

      os.makedirs(storagepath, exist_ok=True)

      # 1. Modell mit LibreCos eigener Methode speichern
      self.model.save(storagepath, model_name="BPR_model")

      # 2. User-Mapping als JSON
      with open(os.path.join(storagepath, "user_mapping.json"), "w") as f:
          json.dump(self.user_id_map, f)

      # 3. Rezeptnamen-Daten
      self.name_df.to_json(
          os.path.join(storagepath, "recipe_names.json"),
          orient="records"
      )

      # 4. Gefilterte Daten
      if self.data_filtered is not None:
          self.data_filtered.to_parquet(
             os.path.join(storagepath, "filtered_data.parquet")
          )

    @classmethod
    def get(cls, storagepath):
        """Lädt gespeicherte Instanz"""
        instance = cls.__new__(cls)
        instance.data_path = None  # Nicht mehr relevant

        # 1. Modell laden
        instance.model = BPR.load(
            path=storagepath,
            model_name="BPR_model",
            data_info=None  # Wird automatisch geladen
        )

        # 2. DataInfo aus dem Modell holen
        instance.data_info = instance.model.data_info

        # 3. User-Mapping laden
        with open(os.path.join(storagepath, "user_mapping.json"), "r") as f:
            instance.user_id_map = json.load(f)

        # 4. Rezeptnamen
        instance.name_df = pd.read_json(
            os.path.join(storagepath, "recipe_names.json"),
            orient="records"
        )

        # 5. Gefilterte Daten
        instance.data_filtered = pd.read_parquet(
            os.path.join(storagepath, "filtered_data.parquet")
        )

        return instance

#----------------------------------------------------------------------------


    def _load_recipe_names(self):
        """Load recipe ID to name mapping"""
        path = kagglehub.dataset_download(self.data_path)
        raw_recipes_path = os.path.join(path, "RAW_recipes.csv")
        self.name_df = pd.read_csv(raw_recipes_path)[["name", "id"]]

    def _rename_and_filter_data(self, interactions_data):
      # Erzeuge explizite Kopie des DataFrames
      df = interactions_data.copy()

      # Spalten umbenennen (ohne inplace)
      df = df.rename(columns={
          "user_id": "user",
          "recipe_id": "item",
          "rating": "label"
      })

      # Spalten filtern
      keep_cols = ["user", "item", "label"]
      df = df[keep_cols]

      # Typkonvertierung mit .loc
      df.loc[:, "label"] = df["label"].astype(int)
      return df

    def _get_recipe_name(self, recipe_id):
        """Helper to get recipe name from ID"""
        name = self.name_df.loc[self.name_df['id'] == recipe_id, 'name']
        return name.values[0] if not name.empty else "Unknown Recipe"

    def import_ratings_csv(self, file_path):
      """Import ratings from CSV and map UUIDs to numeric IDs"""
      try:
          # Load CSV
          df = pd.read_csv(file_path)
          print("CSV erfolgreich geladen:")
          print(df.head())

          # Check required columns
          required = {"uuid", "item_id", "rating"}
          if not required.issubset(df.columns):
              missing = required - set(df.columns)
              raise ValueError(f"Fehlende Spalten: {missing}")

          # Process and map UUIDs
          processed_df = self.__process_ratings(df)

          # Add to data
          self.data_filtered = pd.concat(
              [self.data_filtered, processed_df],
              ignore_index=True
         )
          print(f"{len(processed_df)} neue Bewertungen hinzugefügt.")

      except FileNotFoundError:
          print(f"Datei {file_path} nicht gefunden.")
      except Exception as e:
          print(f"Fehler: {str(e)}")

    def __process_ratings(self, df):
      """Map UUIDs to numeric IDs"""
      # Rename columns
      df = df.rename(columns={
          "uuid": "user",
          "item_id": "item",
          "rating": "label"
      })

      # Determine current max ID from user_id_map
      current_max = max(self.user_id_map.values()) if self.user_id_map else 0

      # Generate new IDs for unknown UUIDs
      new_users = [uuid for uuid in df["user"].unique() if uuid not in self.user_id_map]
      num_new = len(new_users)

      if num_new > 0:
          new_ids = range(current_max + 1, current_max + num_new + 1)
          self.user_id_map.update(zip(new_users, new_ids))

      # Replace UUIDs with numeric IDs
      df["user"] = df["user"].map(self.user_id_map)
      return df


    def __get_score(self,userid,itemid):
     return self.model.predict(userid,itemid)

    def __find_item_by_id(self,recipe_id, items_information):
      df = items_information.loc[items_information["id"] == recipe_id]
      return df['name'].values[0], df['ingredients'].values[0]


In [ ]:
# Beispielaufruf
recommender = RecipeRecommender()
recommender.load_and_preprocess(min_interactions=20)
# Neue Nutzer per CSV importieren
recommender.import_ratings_csv("/content/sample_data/ratings.csv")
recommender.train()
recommender.evaluate()

100%|██████████| 267M/267M [00:01<00:00, 167MB/s]

Extracting files...


CSV erfolgreich geladen:
                                   uuid  item_id  rating
0  df903ba4-a8f3-406f-814d-4420b00ab611    58086     2.0
1  df903ba4-a8f3-406f-814d-4420b00ab611    40621     1.0
2  df903ba4-a8f3-406f-814d-4420b00ab611    85475     0.0
3  df903ba4-a8f3-406f-814d-4420b00ab611    71967     0.0
4  df903ba4-a8f3-406f-814d-4420b00ab611    38584     1.0
75 neue Bewertungen hinzugefügt.
Training start time: 2025-03-25 14:52:27


train: 100%|██████████| 517/517 [00:06<00:00, 75.30it/s]


Epoch 1 elapsed: 6.868s
	 train_loss: 0.6923


eval_listwise: 100%|██████████| 2362/2362 [00:01<00:00, 1397.24it/s]


	 eval log_loss: 0.6926
	 eval roc_auc: 0.5466
	 eval precision@10: 0.0037
	 eval recall@10: 0.0069
	 eval ndcg@10: 0.0176


eval_listwise: 100%|██████████| 2389/2389 [00:01<00:00, 1369.11it/s]


{'loss': 0.6925667028176997,
 'roc_auc': 0.5528957386252877,
 'precision': 0.004646295521138552,
 'recall': 0.009451219641791775,
 'ndcg': 0.02344910293946768}

In [ ]:
recommender.info("df903ba4-a8f3-406f-814d-4420b00ab611")

,user,item,label,name
0,df903ba4-a8f3-406f-814d-4420b00ab611,58086,2.0,dijon beets
1,df903ba4-a8f3-406f-814d-4420b00ab611,40621,1.0,fish cakes fast and simple
2,df903ba4-a8f3-406f-814d-4420b00ab611,85475,0.0,dijon chicken with panko crust
3,df903ba4-a8f3-406f-814d-4420b00ab611,71967,0.0,homemade mayonnaise
4,df903ba4-a8f3-406f-814d-4420b00ab611,38584,1.0,yummy and super easy crock pot oatmeal
5,df903ba4-a8f3-406f-814d-4420b00ab611,50719,1.0,the sweetest blueberry muffins
6,df903ba4-a8f3-406f-814d-4420b00ab611,17987,1.0,quick butternut squash soup
7,df903ba4-a8f3-406f-814d-4420b00ab611,78747,0.0,gallo pinto costa rican rice and beans
8,df903ba4-a8f3-406f-814d-4420b00ab611,26526,0.0,amish white bread
9,df903ba4-a8f3-406f-814d-4420b00ab611,16716,2.0,best ever buckeyes


In [ ]:
def load_items_information():
    path = kagglehub.dataset_download("shuyangli94/food-com-recipes-and-user-interactions")

    recipes_path = os.path.join(path, "RAW_recipes.csv")
    recipes = pd.read_csv(recipes_path)

    return recipes

items_information = load_items_information()

In [ ]:
# Empfehlungen für importierten Nutzer (UUID)
recommendations = recommender.save_recommendations_as_csv(items_information,20,"/content/sample_data/recommendations.csv")

In [ ]:
recommendations

,uuid,item_id,item_title,item_ingredients
0,df903ba4-a8f3-406f-814d-4420b00ab611,50453,teresa s veal or chicken piccata,"['veal scallops', 'flour', 'salt and pepper', ..."
1,df903ba4-a8f3-406f-814d-4420b00ab611,34688,potato kielbasa skillet,"['potatoes', 'kielbasa', 'onion', 'green peppe..."
2,df903ba4-a8f3-406f-814d-4420b00ab611,109536,caesar chicken pasta,"['dry penne pasta', 'cooked chicken', 'caesar ..."
3,df903ba4-a8f3-406f-814d-4420b00ab611,309422,chicken in balsamic barbecue sauce,"['chicken breasts', 'onion', 'olive oil', 'bal..."
4,df903ba4-a8f3-406f-814d-4420b00ab611,216350,linda s cheesecake stuffed strawberries,"['strawberries', 'cream cheese', 'vanilla', ""c..."
...,...,...,...,...
95,4ad3e31c-4a97-4256-8856-361da6ef1ed8,43509,crumb topped banana muffins,"['flour', 'baking soda', 'baking powder', 'sal..."
96,4ad3e31c-4a97-4256-8856-361da6ef1ed8,82102,kittencal s moist cheddar garlic oven fried ch...,"['butter', 'garlic', 'garlic powder', 'seasoni..."
97,4ad3e31c-4a97-4256-8856-361da6ef1ed8,47391,crispy tilapia,"['tilapia fillets', 'flour', 'egg', 'buttermil..."
98,4ad3e31c-4a97-4256-8856-361da6ef1ed8,31078,tarragon chicken casserole,"['boneless skinless chicken breasts', 'onion',..."
